In [2]:
import database as db
import ipywidgets as widgets
from ipywidgets import Button, Layout
from IPython.display import display,clear_output,HTML
from ipywidgets import Button, HBox, VBox
import traceback
import os

dm = db.DataManager('default','./to_annotate/wu.txt')

verbs = None
if os.path.exists('./to_annotate/verb.txt'):
    verb_f = open('./to_annotate/verb.txt','r',encoding='utf-8')
    verbs = verb_f.read().split('\t\n')[:-1]   

args_type = ['arg0','arg1','arg2','arg4','loc','tmp','prp']
arg_list = []
pred_input = widgets.Text(description='predicate',layout=Layout(width='80%'))
for arg_str in args_type:
    arg_list.append(widgets.Text(description=arg_str,layout=Layout(width='80%')))
left_box = VBox([pred_input]+arg_list[0:3],layout=Layout(width='30%'))
right_box = VBox(arg_list[3:],layout=Layout(width='30%'))
box = HBox([left_box,right_box])

def load_data(dm,sentid):
    
    data = dm.data
    punctuation = ['。', '，', '？', '&', '！', '@', '；', '（', '）', '、']
    record = data['id2sent'][sentid]
    record = record.split(' ')
    i = 0
    conversation = ''
    predicate_suggests = []
    for word in record:
        if word in punctuation:
            conversation += word
            i += 1
            continue
        if word == '<SEP>':
            conversation += '<br>'
            continue
        
        new_append = word+'({}) '.format(i)
        if verbs and word in verbs:
            predicate_suggests.append(new_append)
            new_append = '<b>' + new_append + '</b>'
        conversation += new_append
        i += 1
    display(HTML(conversation))
    print('possible predicate:')
    print(predicate_suggests)

def display_all():
    display(box)
    display(button_box)
    print('progress:{}/{}'.format(dm.cur_idx,len(dm.todo)))

def submit_result(b):
    id_ = dm.fetch_next()
    try:
        arg_dict = {}
        pred_idx = int(pred_input.value)
        pred_input.value = ''
        for idx,t in enumerate(arg_list):
            if t.value != '':
                val = t.value.split(',')
                val = [int(i) for i in val]
                if len(val) == 1:
                    val.append(val[0])
                assert val[1] >= val[0]
                arg_dict[args_type[idx]] = val
                t.value =  ''
        id_ = dm.fetch_next()
        dm.save(id_,pred_idx,arg_dict)
        print('just commit: pred:',pred_idx,'arg:',arg_dict)
    except:
        print('error! please examine the input format')
        traceback.print_exc()

def next_sentence(b):
    dm.commit()
    id_ = dm.fetch_next()
    clear_output()
    load_data(dm,id_)
    display_all()
    
def save(b):
    dm.data.sync()
    dm.data.close()
    clear_output()
    print('save done!')
    
def out(b):
    db.output_annotations(dm.data)
    print('output done!')
    
submit_button = widgets.Button(description="Submit",)
next_button = widgets.Button(description='Next Sentence')
save_button = widgets.Button(description='Save and Quit')
out_button = widgets.Button(description='Ouput result to json')
button_box = HBox([submit_button,next_button,save_button,out_button])

out_button.on_click(out)
next_button.on_click(next_sentence)
submit_button.on_click(submit_result)
save_button.on_click(save)

id_ = dm.fetch_next()
load_data(dm,id_)
display_all()

possible predicate:
['喜欢(1) ', '喜欢(6) ', '看(7) ', '推荐(14) ', '给(15) ', '有空(21) ', '去(24) ', '看看(25) ', '是(28) ', '是(36) ', '可以(43) ', '关注(44) ', '能(50) ', '介绍(52) ', '是(59) ', '是(64) ', '去(71) ', '看看(72) ']


progress:0/9232
output done!
